# Read Table
#### 1.
It is necessary to import the libraries, then the wikitable is read by BeautifulSoup library
#### 2.
Each Row is load to matrix, then turn to a Pandas DataFrame, and clean
#### Assumptions
The three Columns has to be in the same order ['PostalCode', 'Borough',  'Neighborhood']  


### Import Libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

from bs4 import BeautifulSoup

print('Libraries imported.')


Libraries imported.


### Read Wikipedia page

In [4]:
#read the Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes 
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
website_url = requests.get(url).text
soup = BeautifulSoup(website_url,'lxml')
table = soup.find('table',{'class':'wikitable sortable'})

#Create a frame
matrix=[]
for tr in table.find_all("tr"):
    td=tr.find_all('td')
    matrix.append([x.text.strip() for x in td])

#Convert to a dataFrame
columns = ['PostalCode', 'Borough',  'Neighborhood']
canada_data = pd.DataFrame(matrix, columns = columns)
canada_data = canada_data[1:]
canada_data = canada_data.loc[canada_data[columns[1]] != "Not assigned"]
canada_data.loc[canada_data[columns[2]] == "Not assigned",columns[2]] = canada_data.loc[canada_data[columns[2]]=="Not assigned",columns[1]]
canada_data = canada_data.groupby([columns[0],columns[1]])[columns[2]].apply(lambda tags: ','.join(tags)).to_frame().reset_index()
canada_data.shape

(103, 3)

# Part two

### Read Coordenates

In [5]:
#Read coordenates for Postal Code
latlng = pd.read_csv('https://cocl.us/Geospatial_data')
latlng.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
#Coordenates with borough and neighboohood
canada_merged = canada_data.join(latlng.set_index('Postal Code'), on='PostalCode')
canada_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
